# PEFT - LoRA

https://huggingface.co/docs/peft/en/developer_guides/lora


**1. 다양한 초기화 전략 (Initialization)**

LoRA 가중치를 어떻게 시작하느냐에 따라 학습 속도와 성능이 달라진다.

* **기본값:** 가중치 A는 Kaiming-uniform, B는 0으로 초기화하여 처음에는 정등 변환(Identity transform) 상태로 시작한다.
* **PiSSA:** 주특이값(Principal singular values)을 사용하여 초기화하며, 일반 LoRA보다 수렴 속도가 빠르고 성능이 우수하다.
* **OLoRA:** QR 분해를 활용하여 베이스 모델의 가중치를 변환하며, 학습 안정성과 수렴 속도를 높인다.
* **EVA:** 입력 활성화 값에 대해 SVD를 수행하여 데이터 기반으로 초기화하며, 레이어별로 랭크(Rank)를 유연하게 할당한다.

**2. 양자화 모델 최적화 (LoftQ & DoRA)**

* **LoftQ:** 양자화된 모델을 미세 조정할 때 발생하는 오차를 최소화하도록 LoRA 가중치를 초기화하는 기술이다.
* **DoRA (Weight-Decomposed Low-Rank Adaptation):** 가중치 업데이트를 크기(Magnitude)와 방향(Direction)으로 분리하여 처리하며, 특히 낮은 랭크에서도 높은 성능을 보여준다.

**3. 효율적인 추론 및 학습 기법**

* **aLoRA (Activated LoRA):** 특정 토큰(invocation tokens)이 나타날 때만 어댑터를 활성화하는 방식이다. 기본 모델과 KV 캐시를 공유할 수 있어 추론 속도를 획기적으로 높일 수 있다.
* **Rank-stabilized LoRA (rsLoRA):** 랭크()의 제곱근에 비례하여 스케일링을 조절함으로써, 높은 랭크를 사용할 때 학습을 안정화시킨다.
* **레이어 복제 (Layer Replication):** 기존 모델의 레이어를 논리적으로 복제하여 모델 크기를 확장하되, 실제 메모리 사용량은 최소화하면서 어댑터를 추가하는 방식이다.

**4. 고급 최적화 및 제어**

* **특수 옵티마이저:** 가중치 A를 고정하고 B만 튜닝하여 메모리를 아끼는 **LoRA-FA**, A와 B에 서로 다른 학습률을 적용해 속도를 2배 높이는 **LoRA+** 등을 지원한다.
* **세부 제어:** 특정 레이어마다 서로 다른 랭크()나 알파() 값을 지정할 수 있는 기능을 제공한다.
* **토큰 학습:** 특정 레이어의 가중치뿐만 아니라, 특정 토큰 임베딩만 선택적으로 학습시키는 기능을 지원한다.

In [ ]:
%pip install -Uq transformers datasets accelerate trl peft hf_transfer pydantic langchain-huggingface

In [ ]:
!nvidia-smi  # GPU 모델명 / 드라이브 버전 / 메모리 사용량 등 확인

In [ ]:
# Local 실행인 경우 아래 코드로 키를 설정
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
HF_TOKEN = os.getenv("HF_TOKEN")

In [ ]:
# Runpod 실행인 경우 아래 코드로 키를 설정
# import os

# OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# HF_TOKEN = os.environ["HF_TOKEN"]

## 데이터셋 로드
https://huggingface.co/datasets/capybaraOh/naver-economy-news2stock

In [ ]:
from datasets import load_dataset  # HuggingFace 데이터셋 로더

# Hub에서 split이 train인 데이터 로드
dataset = load_dataset('capybaraOh/naver-economy-news2stock', split='train')
print(len(dataset))  # 샘플 개수
print(dataset)       # 객체 정보

In [ ]:
dataset[100]  # 101번째 샘플을 확인 (딕셔너리)

## 데이터셋 분할

In [ ]:
test_ratio = 0.2  # 평가셋 비율

train_data = []   # 학습 데이터 리스트
test_data = []    # 테스트 데이터 리스트

data_indices = list(range(len(dataset)))       # 전체 인덱스
test_size = int(len(dataset) * test_ratio)     # 테스트 셋 크기

test_data_indices = data_indices[:test_size]   # 앞부분은 평가셋으로 사용
train_data_indices = data_indices[test_size:]  # 나머지는 학습셋으로 사용

# 학습/평가 데이터셋 형식을 지정하는 함수
def format_data(data):
    # OpenAI / Chat 학습용 messages 포맷 반환
    return {
        'messages': [
            {
                'role': 'system',
                'content': data['system']
            },
            {
                'role': 'user',
                'content': data['user']
            },
            {
                'role': 'assistant',
                'content': data['assistant']
            },
        ]
    }

# 학습 / 평가 인덱스를 변환해 리스트 생성
train_data = [format_data(dataset[i]) for i in train_data_indices]
test_data = [format_data(dataset[i]) for i in test_data_indices]

print(len(train_data))
print(len(test_data))

In [ ]:
print(train_data[100])  # 학습 데이터의 101번째 샘플 확인
print(type(train_data))  # train_data는 list

In [ ]:
from datasets import Dataset  # Huggfing Face Dataset 컨테이너

train_dataset = Dataset.from_list(train_data)  # train_data(list) -> Dataset 변환
test_dataset = Dataset.from_list(test_data)    # test_data(list) -> Dataset 변환

print(train_dataset[100])
print(type(train_dataset))  # train_dataset은 Dataset형

## NCSOFT/Llama-VARCO-8B-Instruct란?
https://huggingface.co/NCSOFT/Llama-VARCO-8B-Instruct


* **기반 모델:** Meta의 Llama-3.1-8B 모델을 기반으로 한다.
* **개발 목적:** 한국어 능력을 극대화하는 동시에 영어 구사 능력도 유지하도록 설계되었다.
* **학습 방법:** 한국어와 영어 데이터셋을 활용한 지속 사전 학습(Continual Pre-training)을 거쳤으며, 이후 지도 미세 조정(SFT)과 직접 선호도 최적화(DPO)를 통해 인간의 선호도에 맞게 정렬되었다.


**SFT에서 한국어능력향상과 동시에 영어능력유지란:**

일반적으로 한국어 데이터를 대량으로 추가 학습시키면 기존에 모델이 가지고 있던 영어 지식이 손상되는 '파괴적 망각(Catastrophic Forgetting)' 현상이 발생한다. 엔씨소프트는 이를 방지하기 위해 **지속 사전 학습(Continual Pre-training)**을 적용했다.

**_1. 데이터 믹스(Data Mixing) 전략:_**

단순히 한국어 데이터만 밀어 넣는 것이 아니라, 모델이 이미 학습했던 영어 데이터와 고품질의 한국어 데이터를 특정 비율로 섞어 학습한다. 이를 통해 기존의 영어 추론 능력을 '복습'하면서 새로운 언어 체계를 '습득'하게 된다.

**_2. 토크나이저 효율화와 임베딩 확장:_**

기존 Llama-3.1의 토크나이저 성능을 유지하면서 한국어 표현력을 높이기 위해 어휘 사전(Vocabulary)을 최적화한다. 영어 토큰 정보는 건드리지 않고 한국어 토큰의 밀도를 높여 두 언어 간의 연결 고리를 강화하는 방식이다.

**_3. 지식 전이(Knowledge Transfer):_**

영어 데이터로 학습된 모델의 강력한 논리적 사고 능력을 한국어로 전이시키는 과정을 거친다.

* **추론 능력 유지:** 수학이나 코딩 같은 논리적 작업은 영어 데이터에서 배운 구조를 그대로 활용한다.
* **언어 정렬:** SFT(지도 미세 조정) 단계에서 동일한 질문을 한국어와 영어로 번급하며 학습시켜, 언어에 상관없이 일관된 답변을 내놓도록 유도한다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM  # 토크나이저 / 생성형 모델 로더
import torch

pretrained_model_name = 'NCSOFT/Llama-VARCO-8B-Instruct'  # 사용할 사전학습 모델 ID

# 사전학습 CausalLM 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name,   # 모델 이름
    dtype = torch.bfloat16,  # 가중치 로딩 dtype(bf16)
    device_map = 'auto'      # 환경에 맞게 CPU/GPU 자동 배치
)

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)  # 해당 모델의 토크나이저 로드

## llama-3 chat template 변환

Llama3 모델은 특정 chat template 형식으로 학습되어, 그 형식을 사용해야 최적 성능을 낼 수 있다.
Chat template을 사용하지 않으면 모델이 대화 구조를 제대로 인식하지 못할 수 있다.
opean_ai 형식의 데이터를 llama-3 형식으로 변환한다.


**LLaMA-3 채팅 포맷**
LLaMA-3 채팅 포맷은 LLaMA-3 계열 챗봇 모델이 대화 내용을 이해하고 답변할 수 있도록 만들어진 입력 데이터 구조입니다.
여러 역할(시스템, 유저, 어시스턴트)의 메시지를 특별한 토큰과 구조로 묶어서 하나의 프롬프트로 합치는 방식입니다.
구조 예시
아래와 같이 대화 흐름을 명확히 구분하는 토큰들이 사용됩니다:

```
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
[시스템 역할 지침]<|eot_id|>
<|start_header_id|>user<|end_header_id|>
[유저 질문]<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
[모델의 답변]<|eot_id|>
```
* <|begin_of_text|> : 전체 프롬프트의 시작을 알리는 토큰
* <|start_header_id|>role<|end_header_id|> : 각 메시지의 역할 구분(시스템, 유저, 어시스턴트 등)
* 각 메시지 끝에 <|eot_id|> : 하나의 메시지 블록이 끝났음을 알림
* 마지막 assistant 블럭은 응답 생성 위치를 가리킨다. apply_chat_template(add_generation_prompt=False)로 설정했더라도 내부 템플릿에는 응답을 받을 자리 표시자로 <|assistant|> 토큰이 남아 있어, "여기서부터 어시스턴트가 답변을 생성해야 한다"는 신호를 제공하는 것임.

**왜 이 포맷이 필요할까?**

* 모델이 **“어디까지가 시스템 안내, 어디서부터가 유저 질문, 어디서부터가 답변인지”** 정확하게 파악할 수 있다.
* 여러 턴(turn)의 대화가 이어질 때도 메시지 경계를 명확히 구분해 혼동 없이 맥락을 유지할 수 있다.
* LLaMA-3 계열 모델은 이런 포맷으로 학습되어 있기 때문에 **실전 파인튜닝/추론 시에도 반드시 이 구조로 입력해야** 기대하는 챗봇 성능을 발휘할 수 있다.

In [ ]:
# apply_chat_template 함수
# - openai 방식의 메시지를 llama3 방식으로 변환
text = tokenizer.apply_chat_template(train_dataset[100]['messages'], tokinize=False)  # messages -> 채팅 프롬프트 문자열 변환
print(text)

### data_collator 함수

* 미니배치(batch) 데이터를 모델이 바로 학습할 수 있는 형태(토큰·마스크·정답)로 변환합니다.
* 특히 아래와 같은 LLaMA-3 채팅 포맷을 쓸 때,
  “어디까지가 질문/어디서부터가 답변(assistant)인지”를 정확히 구분해서
  모델이 정답(답변 부분)만 학습하도록 레이블을 지정합니다.

#### 함수 설명

**1. 프롬프트 생성 (Prompt Construction)**

입력받은 `batch` 데이터는 리스트 내에 여러 메시지(`system`, `user`, `assistant`)를 포함하는 사전(dict) 구조이다.

* Llama 3의 특수 토큰(` <|begin_of_text|>`, `<|start_header_id|>`, `<|eot_id|>`)을 사용하여 모든 대화 내용을 하나의 긴 문자열로 병합한다.
* 각 역할(role)의 시작과 끝을 명확히 구분하여 모델이 대화 맥락을 이해할 수 있도록 구성한다.

**2. 토크나이즈 및 패딩 (Tokenization)**

병합된 문자열 리스트를 `tokenizer`를 통해 숫자 ID(`input_ids`)로 변환한다.

* `padding=True`: 배치 내의 문장들 중 가장 긴 문장을 기준으로 길이를 맞춘다.
* `truncation=True`: `max_length`를 초과하는 데이터는 절단한다.
* `return_tensors="pt"`: PyTorch 텐서 형식으로 결과를 반환한다.

**3. 레이블 생성 및 Loss Masking**

이 함수의 핵심 부분이다. 모델이 '사용자의 질문'이 아닌 **'모델의 답변(assistant)'** 부분에 대해서만 학습하도록 설정한다.

* **-100 값의 의미**: PyTorch의 `CrossEntropyLoss`는 레이블 값이 `-100`인 경우 손실(Loss) 계산에서 제외한다. 이를 통해 모델은 질문 부분을 예측하려고 노력하지 않고, 답변 부분의 정확도에만 집중하게 된다.
* **구간 탐색**: `assistant_tokens`를 기점으로 답변이 시작되는 위치를 찾고, `<|eot_id|>` 토큰이 나오는 지점까지의 인덱스를 추출한다.
* **값 복사**: 해당 구간의 `labels`에만 실제 `input_ids` 값을 복사하여 넣는다.

In [ ]:
# Chat 모델 학습용 데이터 콜레이터: 프롬프트 생성 -> 토크나이저/패딩 -> assistant 구간만 라벨링

# 배치(messages)를 모델 학습 텐서로 변환하는 함수
def data_collator(batch, tokenizer=tokenizer, max_length=8192):
    # 1. 프롬프트 생성
    prompts = []  # 배치프롬프트 문자열을 담을 리스트
    for example in batch:
        prompt = '<|begin_of_text|>'  # 프롬프트 시작
        for msg in example['messages']:  # system/user/assistant 순회
            role = msg['role']
            content = msg['content'].strip()
            # role+content를 템플릿으로 누적
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"
        prompts.append(prompt)  # 완성된 프롬프트를 배치 리스트에 추가
    
    # 2. 토큰처리/패딩/텐서변환
    tokenized = tokenizer(        # 프롬프트를 토큰화해서 텐서로 변환
        prompts,                  # 배치 프롬프트 목록
        truncation = True,        # 최대길이 초과시 자름
        max_length = max_length,  # 최대 토큰 길이
        padding = True,           # 배치 내 최장 길이 기준 패딩
        return_tensors = "pt"     # Pytorch 텐서 반환
    )
    input_ids = tokenized['input_ids']  # 토큰 id 텐서
    attention_mask = tokenized['attention_mask']  # 패딩 마스크 텐서

    # 3. 라벨 생성
    labels = torch.full_like(input_ids, fill_value=-100)  # 기본은 -100(손실 계산에서 제외)

    assistant_header = '<|start_header_id|>assistant<|end_header_id|>'
    assistant_token_id = tokenizer.encode(assistant_header, add_special_tokens=False)  # 헤더의 토큰 패턴
    eot_token = '<|eot_id|>'
    eot_token_id = tokenizer.encode(eot_token, add_special_tokens=False)  # 종료 토큰의 토큰 패턴

    for i, ids in enumerate(input_ids):  # 배치 내 각 샘플별로 라벨 구간 설정
        ids_list = ids.tolist()          # 슬라이싱 비교를 위한 리스트 변환
        # assistant 답변 시작위치 찾기
        # - <|start_header_id|>assistant<|end_header_id|>\n 다음 인덱스부터 답변 시작
        start = None  # 답변 시작 인덱스 초기화
        for idx in range(len(ids_list) - len(assistant_token_id) + 1):  # 헤더 길이만큼 슬라이딩 탐색
            if ids_list[idx: idx + len(assistant_token_id)] == assistant_token_id:  # 헤더 패턴 매칭
                start = idx + len(assistant_token_id)  # 헤더 다음 토큰부터 라벨 시작
                break  # 첫 번째 assistant 구간만 탐색
        # 답변 끝 위치 찾기
        # - <|eot_id|> 까지
        if start is not None:  # assistant 헤더를 찾은 경우
            end = None  # 답변 종료 인덱스 초기화
            # start 이후부터 <|eot_id|> 패턴이 나오는 곳까지를 탐색
            for idx in range(start, len(ids_list) - len(eot_token_id) + 1):
                if ids_list[idx: idx + len(eot_token_id)] == eot_token_id:  # 종료 패턴 매칭
                    end = idx + len(eot_token_id)  # eot까지 포함한 구간 설정
                    break  # 첫 번째 eot 탐색 완료시 종료
        
        labels[i, start:end] = input_ids[i, start:end]  # assistant 답변 구간을 정답 라벨로 복사
    
    return {
        'input_ids': input_ids,            # 모델 입력
        'attention_mask': attention_mask,  # 패딩 마스크
        'labels': labels                   # 손실 계산용 라벨(assistant 구간)
    }

data_collator([train_dataset[0], train_dataset[1]])


### Causal Language Model 파인튜닝: input_ids와 labels 구조 이해

**데이터 구조**
```
input_ids:  [system_tokens..., user_tokens..., assistant_tokens...]  # 전체 시퀀스
labels:     [-100, -100, ..., -100, assistant_tokens...]          # assistant만 학습 대상
```

| 항목 | 내용 |
| --- | --- |
| **Input IDs** | 프롬프트 + 정답 (전체 시퀀스) |
| **Labels** | `-100` (프롬프트 구간) + 정답 토큰 (답변 구간) |
| **결과** | 모델은 입력을 다 보지만, 오직 답변을 맞히는 과정에서만 학습이 일어남 |




> **질문에 해당하는 input_ids에 이미 답이 포함되어 있다!**
>
> **"답이 이미 있는데 어떻게 학습하는가?"**
>
> 모델은 정답을 "보면서" 각 위치에서 올바른 다음 토큰을 예측하는 법을 배운다. 마치 학생이 모범답안을 보며 "이 상황에서는 이렇게 답해야 한다"를 학습하는 것과 같다. 이것이 현대 LLM 파인튜닝의 핵심 메커니즘이다!


**_1. 인과적 언어 모델링 (Causal Language Modeling):_**

LLM(Llama, GPT 등)은 **이전 토큰들을 보고 다음 토큰을 예측**하는 방식으로 학습한다. 따라서 학습 데이터에는 프롬프트와 정답이 모두 포함된 전체 문장이 들어가야 한다.

* **학습 원리:** 모델은 번째 토큰까지를 입력으로 받아 번째 토큰을 예측한다.
* **구조:** `input_ids`가 `[A, B, C, D]`라면, 모델은 내부적으로 `A`를 보고 `B`를, `A, B`를 보고 `C`를 예측하는 과정을 동시에 수행한다.

**_2. Teacher Forcing 기법:_**
```
Position:   [0, 1, 2, 3, 4, 5, 6, 7, 8]
input_ids:  [A, B, C, D, E, F, G, H, I]
labels:     [-100, -100, -100, -100, E, F, G, H, I]
```

학습 과정:
- Position 4: A,B,C,D를 보고 → E 예측
- Position 5: A,B,C,D,E를 보고 → F 예측  
- Position 6: A,B,C,D,E,F를 보고 → G 예측

**_3. Labels와 Loss 계산의 역할:_**

`input_ids`에 정답이 포함되어 있더라도, 모델이 모든 구간에 대해 학습(손실 계산)을 수행하는 것은 아니다. 이때 중요한 역할을 하는 것이 바로 코드에 작성된 **`labels`**이다.

* **-100의 의미:** PyTorch의 `CrossEntropyLoss`는 기본적으로 레이블 값이 `-100`인 위치를 무시(ignore)한다.
* **학습 차단:** 코드에서 프롬프트(User 질문 등) 구간의 레이블을 `-100`으로 설정했기 때문에, 모델이 프롬프트 내용을 예측하며 발생하는 오차는 학습에 반영되지 않는다.
* **학습 집중:** 오직 `assistant`의 답변 구간에 해당하는 `labels`만 실제 `input_ids` 값을 가지므로, 모델은 **"프롬프트가 주어졌을 때 정답을 생성하는 방법"**에 대해서만 가중치를 업데이트한다.


**학습 vs 추론의 차이**

**_학습 시:_**
```
input_ids: <system>당신은 금융분석가</system><user>뉴스내용</user><assistant>분석결과</assistant>
labels:    [-100, -100, ..., -100, 분석결과_토큰들]
```

**_추론 시:_**
```
input:  <system>당신은 금융분석가</system><user>뉴스내용</user><assistant>
output: 분석결과 (모델이 한 토큰씩 생성)
```

In [ ]:
example = train_dataset[128]               # 129번째 샘플
batch = data_collator([example])           # 배치 1개로 콜레이터 적용
print(f'{batch['input_ids'].shape}')       # input_ids 텐서 shape 확인
print(f'{batch['attention_mask'].shape}')  # attention_mask 텐서 shape 확인
print(f'{batch['labels'].shape}')          # labels 텐서 shape 확인

In [ ]:
print(batch['input_ids'][0].tolist())       # 샘플의 input_ids를 리스트로 확인
print(batch['attention_mask'][0].tolist())  # 샘플의 attention_mask를 리스트로 확인
print(batch['labels'][0].tolist())          # 샘플의 labels를 리스트로 확인

In [ ]:
label_ids = [token_id for token_id in batch['labels'][0].tolist() if token_id != -100]  # 정답 토큰(-100 제외)만 추출
text = tokenizer.decode(label_ids)  # 정답 토큰을 문장열로 디코딩
text

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(batch['input_ids'][0].tolist())  # 토큰 ID를 토큰 문자열로 변환

text_tokens = []  # 토큰 ID를 1개씩 디코딩한 문자열을 담을 리스트
for i, token_id in enumerate(batch['input_ids'][0].tolist()):  # 토큰 ID를 순회
    decoded_str = tokenizer.decode([token_id])  # 토큰 1개를 문자열로 디코딩
    text_tokens.append(decoded_str)             # 디코딩 결과를 누적 저장

print(text_tokens)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'token': text_tokens,  # 토큰 (1개씩 디코딩된 문자열)
    'input_ids': batch['input_ids'][0].tolist(),  # 입력 토큰 ID
    'attention_mask': batch['attention_mask'][0].tolist(),  # 패딩 여부(1/0)
    'labels': batch['labels'][0].tolist()  # 정답 라벨(-100 마스킹 포함)
}).transpose()  # 컬럼을 행으로 전치

pd.set_option('display.max_columns', None)  # 출력시 컬럼 생략 없이 표시
df

## PEFT Finetuning - LoRA

* LoRA는 **"Low-Rank Adapter(저랭크 어댑터)"**
* 거대한 대형언어모델(LLM)의 **전체 파라미터를 일일이 미세조정(파인튜닝)하지 않고**,
  **딱 필요한 핵심 부분만 저렴하게 빠르게 학습**하는 최신 파인튜닝.
* **"LLM의 성능은 그대로, 비용/시간/메모리/유지보수는 최소로"** 파인튜닝을 할 수 있게 해주는 AI 실무에서 가장 중요한 기법 중 하나이다.

**왜 LoRA가 등장했을까?**

* GPT, Llama, DeepSeek 같은 대형언어모델은 **파라미터(매개변수) 수가 수십억\~수조 개**나 된다.
* 이런 모델을 파인튜닝하려면 **막대한 GPU 메모리와 시간, 저장 공간**이 필요.
* 하지만, 실제로 특정 태스크에 맞게 모델을 조정할 때 **전체를 다 바꿀 필요가 없다.**
* 대부분의 정보는 기존 모델에 이미 들어있고,
  **특정 입력(질문)과 특정 출력(답변)의 관계만 살짝 조정**해주면 충분하다.

**LoRA의 원리**

* 기존 대형 모델의 핵심 연산(주로 "곱셈" 부분)에
  **작고 얇은 "보조 네트워크(어댑터 레이어)"**를 덧붙인다.
* 전체 모델은 거의 건드리지 않고,
  **이 어댑터 레이어의 파라미터만 새로 추가해서 학습**
* 학습이 끝나면,

  * 원본 모델은 그대로
  * 어댑터(작은 추가 파라미터)만 별도로 저장하면 끝!
* 추론할 땐 **원본 모델 + LoRA 어댑터**를 합쳐서 쓸 수 있다.

**LoRA의 장점**

* **파인튜닝 비용(시간, 메모리, 저장 용량)이 압도적으로 절약**된다.
* 7B, 13B, 70B 등 대형 모델도
  **일반 GPU(24GB/48GB)로도 쉽게 파인튜닝**이 가능하다.
* **동일한 원본 모델에 다양한 LoRA 어댑터만 바꿔 끼우며
  다양한 분야별 파인튜닝 결과를 쉽게 쓸 수 있다.**

**LoRA와 기존 방식의 비교**

* **기존 파인튜닝:**
  전체 파라미터(수십\~수백 GB)를 새로 저장/관리/학습 → 비효율적
* **LoRA:**
  원본은 그대로 두고,
  변화가 필요한 부분(수 MB\~수십 MB)만 별도로 학습/저장


**실전에서의 활용 예시**

* 번역 LoRA, 요약 LoRA, 감정분석 LoRA 등
  **하나의 원본 모델에 여러 용도별 어댑터를 저장/관리**할 수 있다.
* **A100 80GB, 3090, T4 등 다양한 GPU 환경에서도
  고성능 LLM 튜닝이 매우 쉽게 가능하다.**

In [ ]:
from peft import LoraConfig, get_peft_model  # LoRA 설정 / PEFT 적용 함수

lora_config = LoraConfig(
    r = 8,               # 저랭크 행렬 rank
    lora_alpha = 32,     # LoRA 스케일 계수 (alpha/r)
    lora_dropout = 0.1,  # 드롭아웃 비율
    bias = 'none',       # bais 학습 안함
    target_modules = ['q_proj', 'v_proj'],  # LoRA를 주입할 모듈
    task_type = 'CAUSAL_LM'  # 작업 유형(생성)
)

model = get_peft_model(model, lora_config)  # 기존 모델에 LoRA 어댑터 적용
model.print_trainable_parameters()          # 학습 가능한 파라미터 수/비율 출력

In [ ]:
# SFTConfig
from trl import SFTConfig  # TRL SFT 학습 설정 클래스

hub_model_id = 'capybaraOh/Llama-VARCO-8b-news2stock-analyzer'  # 학습 완료 후 업로드할 Hub 모델 ID

sft_config = SFTConfig(  # SFT 학습 하이퍼파라미터/저장/로그 설정
    output_dir="Llama-VARCO-8b-news2stock-analyzer", # 학습 완료된 모델과 체크포인트가 저장될 경로이다.
    num_train_epochs=3,                              # 전체 데이터셋을 반복 학습할 횟수(Epoch)이다.
    per_device_train_batch_size=2,                   # 각 GPU(장치)당 한 번에 처리할 데이터 샘플의 개수이다.
    gradient_accumulation_steps=2,                   # 그래디언트를 2번 누적한 후 가중치를 업데이트한다. (실제 배치 크기 = 2 * 2 = 4 효과를 낸다.)
    gradient_checkpointing=True,                     # VRAM 절약을 위해 중간 활성화 값을 저장하지 않고 역전파 시 재계산하는 설정이다.
    optim="adamw_torch_fused",                       # 최적화 알고리즘 설정이다. fused 버전은 CUDA에서 더 빠르다.
    logging_steps=10,                                # 10 스텝마다 학습 로그(Loss 등)를 출력한다.
    save_strategy="steps",                           # 체크포인트 저장 기준을 'steps'(스텝 수)로 설정한다. (옵션: 'epoch')
    save_steps=100,                                  # 100 스텝마다 모델 체크포인트를 저장한다.
    bf16=True,                                       # BF16(Brain Float 16) 정밀도를 사용하여 메모리를 아끼고 연산 속도를 높인다. (Ampere GPU 이상 권장)
    learning_rate=1e-4,                              # 학습률(Learning Rate)이다. 가중치 업데이트의 크기를 결정한다.
    max_grad_norm=0.3,                               # 그래디언트 클리핑 임계값이다. 그래디언트 폭주를 막아 학습 안정성을 높인다.
    warmup_ratio=0.03,                               # 전체 학습 단계의 3% 동안 학습률을 서서히 올리는 웜업(Warmup)을 수행한다.
    lr_scheduler_type="constant",                    # 학습률 스케줄러 타입이다. 여기서는 학습률을 변동 없이 상수로 유지한다.
    push_to_hub=True,                                # 학습이 끝나면 Hugging Face Hub에 모델을 자동으로 업로드한다.
    hub_model_id=hub_model_id,                       # Hub에 업로드될 때 사용될 저장소(Repository) ID이다.
    hub_token=True,                                  # Hub 업로드를 위해 인증 토큰을 사용한다.
    remove_unused_columns=False,                     # 데이터셋에서 모델의 forward 메서드 시그니처에 없는 컬럼을 자동으로 삭제하지 않도록 한다.
    dataset_kwargs={"skip_prepare_dataset": True},   # 데이터셋 처리 과정(packing 등)을 건너뛰도록 하는 설정이다.
    report_to=[],                                    # 학습 기록을 전송할 툴(WandB, Tensorboard 등)을 지정한다. 빈 리스트는 기록하지 않음을 의미한다.
    label_names=["labels"],                          # 손실(Loss) 계산 시 정답(Target)으로 사용할 데이터셋의 컬럼 이름이다.
)

In [ ]:
from trl import SFTTrainer  # SFT 학습용 Trainer

# SFT 학습 객체 생성
trainer = SFTTrainer(
    model = model,                  # 학습할(LoRA 적용된) 모델
    args = sft_config,              # SFT 학습 설정
    train_dataset = train_dataset,  # 학습 데이터셋
    data_collator = data_collator   # 배치 텐서 생성 함수
)

trainer.train()

## 평가

In [ ]:
# 테스트셋 messages에서 프롬프트/정답(assistant) 텍스트 분리
prompt_list = []   # 프롬프트(assistant메시지 이전) 저장 리스트
label_list = []    # 정답 (assistant 답변) 저장 리스트

for messages in test_dataset['messages']:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)  # 채팅 템플릿 문자열로 변환
    # assistant 시작 전까지는 프롬프트로 사용 + assistant 헤더만 프롬프트 끝에 붙여준다.
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + \
        '<|start_header_id|>assistant<|end_header_id|>\n'
    # assistant 답변(종료 토큰 전)만 추출
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    
    prompt_list.append(input)
    label_list.append(label)

In [ ]:
prompt_list[100]

In [ ]:
label_list[100]

### 추론모델 - 런타임결합
1. lora모델을 로드
2. base모델 + lora adapter 세팅

In [ ]:
from peft import AutoPeftModelForCausalLM  # Hub에 저장된 PEFT 모델 로더
from transformers import AutoTokenizer, pipeline  # 토크나이저 / 파이프라인 생성
import torch

peft_model_id = hub_model_id

finetuned_model = AutoPeftModelForCausalLM.from_pretrained(  # 파인튜닝 된 PEFT 모델 로드
    peft_model_id,
    dtype = torch.bfloat16,  # bf16 로드
    device_map = 'auto'      # CPU/GPU 자동 배치
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)  # 같은 repo에서 토크나이저 로드
pipe = pipeline('text-generation', model=finetuned_model, tokenizer=tokenizer)  # 텍스트 생성 파이프라인
pipe

In [ ]:
eos_token = tokenizer('<|eot_id|>', add_special_tokens=False)['input_ids'][0]  # <|eot_id|>의 토큰ID 추출
eos_token

In [ ]:
# 프롬프트를 입력받으면 assistant 생성 결과만 반환하는 함수
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)  # 응답을 생성
    assistant_start = len(prompt)  # 생성 결과에서 프롬프트 길이만큼은 입력 구간
    return outputs[0]['generated_text'][assistant_start:].strip()  # 프롬프트 이후(생성된 부분)만 잘라서 반환

for prompt, label in zip(prompt_list[10:13], label_list[10:13]):  # 10~12 샘플 비교
    print(f'[prompt]\n{prompt}')
    print(f'[label]\n{label}')
    print(f'[response]\n{test_inference(pipe, prompt)}')  # 모델 생성 응답 출력
    print('=' * 100)

## 추론

In [ ]:
# 뉴스 본문을 넣으면 모델의 분석 응답 텍스트를 반환하는 함수
def inference(news):
    messages = [
        {'role': 'system', 'content': '''
당신은 금융/경제 뉴스의 핵심내용을 요약해 설명하고,
특정 상장 종목에 미치는 긍정/부정 영향여부, 이유, 근거를 분석하는 금융/경제 분석 전문가입니다.

다음 출력지시사항을 지켜주세요.
1. 뉴스와 종목간의 연관성을 발견할 수 없다면:
    - stock_related를 False로 작성하세요.
    - summary에 뉴스의 요약을 작성하세요.
2. 뉴스와 종목간의 연관성을 발견했다면:
    - stock_related를 True로 작성하세요.
    - summary에 뉴스의 요약을 작성하세요.
    - 긍정영향이 예상되는 종목이 있다면, positive_stocks, positive_keywords, positive_reasons를 작성하세요.
    - 부정영향이 예상되는 종목이 있다면, negative_stocks, negative_keywords, negative_reasons를 작성하세요.
    - 값이 없는 경우 빈 문자열(''), 빈 리스트([])로 작성하세요.
'''},  # 시스템 지시문
        {'role': 'user', 'content': news}  # 사용자 입력(뉴스)        
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)  # messages를 채팅 프롬프트 문자열로 변환
    # 파이프라인으로 텍스트 생성 수행
    outputs = pipe(
        prompt,  # 변환된 프롬프트
        max_new_tokens = 1024,    # 생성 최대 토큰 수
        eos_token_id = eos_token, # 종료 토큰 ID
        do_sample = False         # False: Greedy, True: 샘플링 적용
    )
    assistant_start = len(prompt)  # 프롬프트 길이 계산
    return outputs[0]['generated_text'][assistant_start:].strip()  # 프롬프트 이후(생성된 부분)만 잘라서 반환


In [ ]:
# 뉴스 1건을 inference 함수로 분석
news = '''
강훈식 청와대 비서실장이 이끄는 캐나다 방산 특사단 출장 기간 현지에서 한·캐나다 자동차 포럼이 열린다. 한국과 캐나다의 자동차 산업 협력에 대한 논의가 이뤄질 전망이다. 이자리에는 정의선 현대자동차그룹 회장 등 주요 경영진도 참석할 것으로 알려졌다.

26일 자동차 업계 및 정부 관계자에 따르면 정 회장과 장재훈 부회장은 이번주 캐나다에서 열리는 ‘한국·캐나다 자동차 산업 협력 포럼’에 참석한다. 이 행사에는 김정관 산업통상자원부 장관과 멜라니 졸리 캐나다 산업부 장관 등 주요 인사가 모여 자동차 산업 협력 방안에 대해 논의할 예정이다. 정 회장이 현지 일정상 참석이 어려울 경우 장재훈 현대차 부회장만 참석할 가능성도 남아있다.

정 회장은 현지에서 구체적인 캐나다 투자 방안을 공개할 예정이다. 현대차는 캐나다 자원 등 장점을 활용해 수소 분야를 포함한 다양한 협력 방안을 검토 중인 것으로 알려졌다. 캐나다 정부가 요구해 온 ‘전기차 전용 공장 건설’은 투자 명단에서 제외하기로 가닥이 잡혔다. 북미 시장 공략을 위해 지난해 초 미국 조지아주에 완공한 메타플랜트아메리카(HMGMA)와의 중복 투자를 피하기 위해서다.

정 회장은 60조원 규모의 캐나다 초계 잠수함 사업(CPSP) 수주를 지원하기 위해 캐나다 출장길에 올랐다. 정 회장이 전면에 나선 배경에는 ‘절충교역’이 있다. 절충교역은 대규모 방산 계약을 발주하는 국가가 수주국에 현지 투자나 기술이전, 공급망 구축 등을 요구하는 방식이다. 캐나다 정부는 3000t급 디젤 잠수함 12척을 도입하는 대가로 현대차의 현지 투자를 강력히 희망해 왔다. 캐나다는 한국과 경쟁 상대인 독일 측에도 폭스바겐의 현지 생산 확대를 입찰 조건으로 제시한 것으로 전해졌다. 현대차는 캐나다에 생산 시설이 없는 반면 독일 폭스바겐은 배터리 셀 공장을 건설 중이다.

이번 수주전의 성패는 향후 30년간의 유지·보수·정비(MRO) 시장 주도권과 직결된다. 이 사업은 디젤 잠수함 최대 12척을 건조하는 프로젝트다. 건조비용(약 20조원)에 도입 후 30년간 유지·보수·운영(MRO) 비용까지 포함하면 최대 60조원까지 규모가 커질 전망이다.
'''
inference(news)  # news 문자열을 입력으로 넣어 모델 분석 결과(assistant 응답)를 생성

In [ ]:
# 뉴스 1건을 inference 함수로 분석
news = '''
달러당 원화값이 26일 전 거래일 대비 20원 가까이 급등하며 1440원대를 이어가고 있다.

이날 서울외환시장에서 원화값은 전 거래일보다 19.7원 오른 1446.1원에 출발해 오전 10시25분 현재 1446.2원에 거래되고 있다.

미국과 일본 외환당국의 시장 개입 가능성이 거론되면서 엔화가 급등한 점이 원화 강세로 이어졌다는 분석이 나온다. 일본은행은 최근 외환시장 개입에 앞서 주요 은행을 상대로 거래 상황을 점검하는 ‘레이트 체크’를 실시한 것으로 전해졌다. 미국 뉴욕 연방준비은행도 미 재무부 지시에 따라 레이트 체크에 나섰다는 보도가 나왔다.

다카이치 사나에 일본 총리는 “투기적이고 비정상적인 움직임에 필요한 모든 조처를 할 것”이라고 밝힌 바 있다. 이에 따라 지난주 달러당 160엔에 육박했던 달러당 엔화값은 155엔대 초반까지 상승했다. 현재 엔화값은 전 거래일 대비 0.50% 오른 155.04엔이다.

한편 이날 열리는 국민연금 기금운용위원회에서 환헤지 전략 등이 논의될 예정이어서 원화값에 어떤 영향을 미칠지도 주목된다.
'''
inference(news)  # news 문자열을 입력으로 넣어 모델 분석 결과(assistant 응답)를 생성

### Base모델과 비교

In [ ]:
# LoRA 파인튜닝 전(Base) vs 파인튜닝 후(LoRA) 모델 답변 비교
from transformers import AutoModelForCausalLM, pipeline  # 모델 로드/파이프라인 생성

base_model_id = "NCSOFT/Llama-VARCO-8B-Instruct"  # 베이스(파인튜닝 전) 모델 ID
base_model = AutoModelForCausalLM.from_pretrained(  # 베이스 모델 로드
    base_model_id,  # 모델 ID
    dtype=torch.bfloat16,  # BF16 로드
    device_map="auto",  # CPU/GPU 자동 배치
)
base_pipe = pipeline("text-generation", model=base_model, tokenizer=tokenizer)  # 베이스 모델 파이프라인 생성

for idx, (prompt, label) in enumerate(zip(prompt_list[10:13], label_list[10:13])):  # 샘플 3개만 비교
    print(f"[샘플 {idx + 1}]")  # 샘플 번호 출력
    base_resp = test_inference(base_pipe, prompt)  # 베이스 모델 응답 생성
    lora_resp = test_inference(pipe, prompt)  # LoRA 모델 응답 생성
    print(f"  [Base - 파인튜닝 전]\n{base_resp}")  # 베이스 모델 응답 출력
    print(f"  [LoRA - 파인튜닝 후]\n{lora_resp}")  # LoRA 모델 응답 출력
    print(f"  [Label]\n{label}")  # 정답(레이블) 출력
    print("-" * 50)  # 구분선 출력

In [ ]:
# HuggingFace에 자동으로 업로드 되지 않을 경우 수동으로 업로드
# import os
# from huggingface_hub import HfApi  # Hub API 사용

# repo_id = 'capybaraOh/naver-economy-news2stock'      # 업로드할 모델 REPO ID
# local_dir = "./Llama-VARCO-8b-news2stock-analyzer"   # 로컬 모델 폴더 경로

# api = HfApi(token=os.environ["HF_TOKEN"])  # Hub 인증 토큰으로 api 객체 생성
# api.create_repo(repo_id=repo_id, repo_type='model', exist_ok=True)  # repository가 없으면 생성 (있으면 그대로 사용)

# # 로컬 폴더를 Hub에 업로드
# api.upload_folder(
#     folder_path = local_dir,  # 업로드할 로컬 폴더
#     repo_id = repo_id,        # 대상 repository
#     repo_type = 'model',      # 모델 repository에 업로드
#     ignore_patterns = ['chekckpoint-*', '**/checkpoint-*'],  # 체크포인트 폴더들은 제외
#     commit_message = "Upload final LoRA adapter (without checkpoints)"  # 커밋 메시지
# )